In [40]:
%reload_ext autoreload
%autoreload 2

# test model loading and vocab loading
import torch
from transformer.transformer import Transformer
import torch.nn as nn
from trainer import Trainer
from data.translation_data import TranslationData
import sentencepiece as spm

# Test Loading a model, vocabulary and validation

In [ ]:
# create device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# load best model
model_path = './checkpoints/small_model_de3/best_model.pt'
checkpoint = torch.load(model_path, map_location=device)

# load hypter parameters
args = checkpoint['args']

# get tokenizers form sentence piece
sp_tokenizer = spm.SentencePieceProcessor()
if args['tgt_lang'] == 'fr':
    sp_tokenizer.load('data/spm_en_fr.model')
elif args['tgt_lang'] == 'de':
    sp_tokenizer.load('data/spm_en_de.model')
else:
    raise ValueError("only fr and de target languages supported")

print(f"SP piece size ('vocab size'): {sp_tokenizer.get_piece_size()}")
model = Transformer(vocab_size=sp_tokenizer.get_piece_size(), d_model=args['d_model'], n_heads=args['n_heads'],
                       max_len=args['max_len'], dropout_rate = args['dropout_rate'],
                       encoding_type='sinusoidal', hidden_ff_d=args['d_model']*4,
                       num_encoder_layers=args['num_encoder_layers'],
                       num_decoder_layers=args['num_encoder_layers']).to(device=device)

# load dataset
data_module = TranslationData(src_lang='en', tgt_lang='de', batch_size=args['batch_size'],
                              max_len=args['max_len'], tokenizer=sp_tokenizer, small_subset=True)
data_module.prepare_data()
# get validation loader
_, valid_loader, _ = data_module.get_dataloaders()

# create a trainer object for inference
loss_fn = nn.CrossEntropyLoss(ignore_index=data_module.special_tokens['<pad>'])
trainer = Trainer(model=model, val_loader=valid_loader, loss_fn=loss_fn, tokenizer=sp_tokenizer)
trainer.load_checkpoint(path=model_path)
# run validation only
val_loss, bleu_score = trainer.validate()
print(f"Val Loss: {val_loss:.04f} | BLEU Score: {bleu_score:.02f}")

SP piece size ('vocab size'): 32000
Loading dataset...
Data Loaders ready
Cuda available: True


Val Loss: 3.9293 | BLEU Score: 5.10


# Test inference

In [42]:
# take a batch from the validation loader
src_batch, tgt_batch = next(iter(valid_loader))

# send to device
src_batch = src_batch.to(device)
src_batch_cpu = src_batch.cpu().tolist()
# run inference
src_sentences = trainer.decode_ids(id_sequences=src_batch,)
translated_sentences = trainer.infer(src=src_batch, type='greedy')
translated_sentences_beam = trainer.infer(src=src_batch, type='beam')

# Print some translations
for idx in range(5):  # first 5 examples+
    print(f'Source sentences: {src_sentences[idx]}')
    print(f"Predicted Translation Greedy: {translated_sentences[idx]}")
    print(f"Predicted Translation Beam: {translated_sentences_beam[idx]}")
    print("="*50)

Source sentences: Last year I showed these two slides so that demonstrate that the arctic ice cap, which for most of the last three million years has been the size of the lower 48 states, has shrunk by 40 percent.
Predicted Translation Greedy: L'année dernière fois que j'ai été prise de la taille de la taille de la taille de la taille de la taille de la taille de la taille de la taille de la taille de la taille de la taille de la taille de la taille
Predicted Translation Beam: L'année dernière fois que j'ai eu l'année dernière fois que ces deux millions d'année dernière fois que les deux millions de l'année dernière fois que les deux millions d'année dernière fois que les deux millions d'année
Source sentences: But this understates the seriousness of this particular problem because it doesn't show the thickness of the ice.
Predicted Translation Greedy: Mais ce problème de la glace, parce que ce problème ne se passe pas la glace.
Predicted Translation Beam: Mais ce problème de la glace,